# Análisis Avanzado de Clustering Estudiantil - TFM

Este notebook centraliza la experimentación final de clustering para el proyecto de analítica educativa OULAD. El objetivo es comparar la calidad de segmentación entre clústeres generados sobre **PCA** frente a los generados sobre el espacio latente del **Autoencoder (AE)**, utilizando diversas técnicas de agrupamiento.

---
**Autor:** TFM Student  
**Última actualización:** 2026-02-17

## 1. Configuración del Entorno y Constantes Globales

En esta sección definimos los parámetros que rigen todo el notebook (rutas, semillas, parámetros educativos). Centralizar esto aquí nos permite cambiar el curso de la investigación (ej. cambiar el número de clústeres) desde una única celda.

In [ ]:
# =========================
#         COMUNES
# =========================
import os
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from IPython.display import display, Markdown

# Algoritmos y Métricas
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from scipy.cluster.hierarchy import dendrogram, linkage

# Silenciar ruidos de sistema
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
warnings.filterwarnings("ignore")

# Rutas del proyecto
try:
    from educational_ai_analytics.config import EMBEDDINGS_DATA_DIR, FEATURES_DATA_DIR, W_WINDOWS
    ROOT = Path(EMBEDDINGS_DATA_DIR)
    TARGET_PATH = FEATURES_DATA_DIR / "training" / "target.csv"
    WS_TO_PLOT = list(W_WINDOWS)
except ImportError:
    ROOT = Path("/workspace/TFM_education_ai_analytics/data/4_embeddings")
    TARGET_PATH = Path("/workspace/TFM_education_ai_analytics/data/3_features/training/target.csv")
    WS_TO_PLOT = [12, 18, 24]

# Parámetros Globales
SPLIT = "training"
K_GLOBAL = 5
SEED = 42
MAX_POINTS_VIZ = 15000
MAX_POINTS_STATS = 5000

# Paleta de Colores y Etiquetas OULAD
MAP_NUM_TO_STR = {3: "Distinction", 2: "Pass", 1: "Fail", 0: "Withdrawn"}
OUTCOMES = ["Distinction", "Pass", "Fail", "Withdrawn"]
OUTCOME_COLORS = {
    "Distinction": "#f1c40f", 
    "Pass": "#4c78a8", 
    "Fail": "#e45756", 
    "Withdrawn": "#f4a261"
}

def safe_read_csv(path: Path) -> pd.DataFrame:
    if not path.exists(): return pd.DataFrame()
    return pd.read_csv(path, index_col=0).replace([np.inf, -np.inf], np.nan).fillna(0.0)

# Carga inicial de resultados académicos
labels_df = pd.read_csv(TARGET_PATH, index_col=0)
y_outcome = labels_df["final_result"].map(MAP_NUM_TO_STR)
global_success_rate = (labels_df["final_result"] >= 2).mean() * 100

print(f"✅ Configuración cargada. Trabajando con semanas: {WS_TO_PLOT}")
print(f"📌 Tasa de éxito global: {global_success_rate:.1f}%")

## 2. Análisis del Número Óptimo de Clústeres (K)

Antes de aplicar cualquier algoritmo, necesitamos una base empírica para elegir la cantidad de grupos. Aquí usamos dos criterios:
1. **Método del Codo (Elbow)**: Buscamos el punto de saturación de la inercia.
2. **Silhouette Score**: Evaluamos la cohesión y separación media.

In [ ]:
# CONFIG-SECTION: ELBOW
K_RANGE = range(2, 11)

for emb_type in ["pca", "ae"]:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5), constrained_layout=True)
    for W in WS_TO_PLOT:
        df = safe_read_csv(ROOT / SPLIT / f"upto_w{W:02d}" / f"{emb_type}_latent.csv")
        if df.empty: continue
        if len(df) > MAX_POINTS_STATS: df = df.sample(n=MAX_POINTS_STATS, random_state=SEED)
        X = StandardScaler().fit_transform(df.values)
        
        inerts, silhs = [], []
        for k in K_RANGE:
            km = KMeans(n_clusters=k, n_init=10, random_state=SEED).fit(X)
            inerts.append(km.inertia_)
            silhs.append(silhouette_score(X, km.labels_))
        
        ax1.plot(K_RANGE, inerts, marker='o', label=f"W{W}")
        ax2.plot(K_RANGE, silhs, marker='s', label=f"W{W}")
    
    ax1.set_title(f"Elbow Method ({emb_type.upper()})"); ax1.legend()
    ax2.set_title(f"Silhouette Score ({emb_type.upper()})"); ax2.legend()
    plt.show()

## 3. Evaluando la Mixtura de Gaussianas (GMM)

El GMM nos permite un modelado probabilístico más flexible. Para elegir el número de componentes, nos basamos en los criterios de información **BIC (Bayesian Information Criterion)** y **AIC (Akaike)**. El modelo con el BIC más bajo suele ser el que mejor balancea el ajuste frente a la complejidad.

In [ ]:
# CONFIG-SECTION: GMM SELECTION
COV_TYPE = "diag"

for emb_type in ["pca", "ae"]:
    fig, ax = plt.subplots(1, 1, figsize=(10, 5))
    for W in WS_TO_PLOT:
        df = safe_read_csv(ROOT / SPLIT / f"upto_w{W:02d}" / f"{emb_type}_latent.csv")
        if df.empty: continue
        X = StandardScaler().fit_transform(df.sample(min(len(df), 5000), random_state=SEED))
        bics = [GaussianMixture(k, covariance_type=COV_TYPE, random_state=SEED).fit(X).bic(X) for k in K_RANGE]
        ax.plot(K_RANGE, bics, marker='o', label=f"W{W}")
    ax.set_title(f"BIC Score (GMM {COV_TYPE}) - {emb_type.upper()}")
    ax.set_ylabel("BIC Value (Lower is better)"); ax.legend()
    plt.show()

## 4. Comparativa Cuantitativa de Algoritmos

En esta sección generamos una comparativa directa entre los tres grandes enfoques: **K-Means** (Particional), **GMM** (Probabilístico) y **Jerárquico** (Ward). Esta tabla nos permite contrastar científicamente cuál es el mejor método para segmentar en cada espacio de representación.

In [ ]:
# CONFIG-SECTION: METRICS TABLE
HIER_LINKAGE = "ward"

all_metrics = []
for W in WS_TO_PLOT:
    for emb in ["pca", "ae"]:
        df = safe_read_csv(ROOT / SPLIT / f"upto_w{W:02d}" / f"{emb}_latent.csv")
        if df.empty: continue
        X = StandardScaler().fit_transform(df.sample(min(len(df), MAX_POINTS_STATS), random_state=SEED))
        
        # Ajustar modelos
        m_km = KMeans(K_GLOBAL, n_init=10, random_state=SEED).fit_predict(X)
        m_gmm = GaussianMixture(K_GLOBAL, covariance_type="diag", random_state=SEED).fit_predict(X)
        m_hier = AgglomerativeClustering(K_GLOBAL, linkage=HIER_LINKAGE).fit_predict(X)
        
        for name, labs in [("K-Means", m_km), ("GMM", m_gmm), ("Jerárquico", m_hier)]:
            all_metrics.append({
                "Semana": W, "Embedding": emb.upper(), "Modelo": name,
                "Silueta": silhouette_score(X, labs),
                "CH-Score": calinski_harabasz_score(X, labs),
                "DB-Index": davies_bouldin_score(X, labs)
            })

df_res = pd.DataFrame(all_metrics)
for emb in ["PCA", "AE"]:
    display(Markdown(f"### 🏆 Resultados para: **{emb}**"))
    display(df_res[df_res["Embedding"] == emb])

## 5. Dashboard de Resultados y Desglose Educativo

Finalmente, visualizamos los clústeres. El orden de los grupos se ajusta automáticamente por su tasa de abandono (**Withdrawn**), de modo que el "Riesgo 0" sea siempre el perfil más crítico. Esto nos permite comparar visualmente la pureza de los grupos entre PCA y AE.

In [ ]:
# CONFIG-SECTION: VISUALIZATION DASHBOARD
MODEL_TO_PLOT = "GMM" # Opciones: "GMM", "K-Means", "Jerárquico"

nrows = len(WS_TO_PLOT)
fig, axes = plt.subplots(nrows, 4, figsize=(22, 6 * nrows), constrained_layout=True)
if nrows == 1: axes = np.expand_dims(axes, axis=0)

for i, W in enumerate(WS_TO_PLOT):
    for j, emb_type in enumerate(["pca", "ae"]):
        df_emb = safe_read_csv(ROOT / SPLIT / f"upto_w{W:02d}" / f"{emb_type}_latent.csv")
        df_join = df_emb.join(y_outcome.rename("outcome"), how="inner")
        if len(df_join) > MAX_POINTS_VIZ: df_join = df_join.sample(MAX_POINTS_VIZ, random_state=SEED)
        
        X = StandardScaler().fit_transform(df_join.drop(columns=["outcome"]).values)
        
        # Clustering (usamos el seleccionado)
        if MODEL_TO_PLOT == "GMM": labs = GaussianMixture(K_GLOBAL, random_state=SEED).fit_predict(X)
        else: labs = KMeans(K_GLOBAL, random_state=SEED).fit_predict(X)
        
        # Reordenar por riesgo (Withdrawn rate)
        tmp = pd.DataFrame({"cl": labs, "res": df_join["outcome"]})
        rates = tmp.groupby("cl")["res"].apply(lambda x: (x == "Withdrawn").mean()).sort_values(ascending=False)
        map_risk = {old: f"R{idx}" for idx, old in enumerate(rates.index)}
        tmp["group"] = tmp["cl"].map(map_risk)
        order_g = [map_risk[cl] for cl in rates.index]

        # Scatter (PCA view)
        Z = PCA(2).fit_transform(X)
        ax_sc = axes[i, j*2]
        for out in OUTCOMES:
            m = df_join["outcome"] == out
            ax_sc.scatter(Z[m, 0], Z[m, 1], s=4, c=OUTCOME_COLORS[out], alpha=0.5)
        ax_sc.set_title(f"W{W} {emb_type.upper()} | Scatter (Outcome)"); ax_sc.axis('off')

        # Bars (Dashboard)
        ax_bar = axes[i, j*2+1]
        tab = pd.crosstab(tmp["group"], tmp["res"], normalize='index') * 100
        for o in OUTCOMES: 
            if o in tab.columns: ax_bar.bar(order_g, tab.reindex(order_g)[o], label=o, color=OUTCOME_COLORS[o])
        ax_bar.axhline(global_success_rate, color='k', ls='--')
        ax_bar.set_title(f"Frecuencias W{W} {emb_type.upper()}")

plt.show()